In [ ]:
import csv
import pandas as pd
import json
import itertools
import numpy as np
import json
from statsmodels.stats.contingency_tables import mcnemar
from scipy.stats import fisher_exact

tad_path = 'tad/GSE63525_K562_{}_TAD.txt'
tss_path = 'refTSS_v3.1_human_annotation.txt'
annot_path = 'refTSS_v3.1_human_coordinate.hg38.bed'
cont_path = "K562_tss_survival.json"
result_path = 'GWAS_results.txt'
pair_path = "pairs_by_chrom.json"

tads = []

tad_file = open('GSE63525_K562_domains.bed')
header = ['Start', 'End', 'Cen']

chrom_tad_path = 'tad/GSE63525_K562_chr{}_TAD.txt'

sqtl_path = 'LAML_CompleteSurvival_sQTLs.csv'

RESOLUTION = 10000
CHROMOSOMES = ["chr1","chr2","chr3","chr4","chr5","chr6","chr7","chr8","chr9","chr10","chr11","chr12","chr13","chr14","chr15","chr16","chr17","chr18","chr19","chr20","chr21","chr22","chrX","chrY"]

In [ ]:
def symbol_to_tss_id(symbol):
    return [id for (sym,id) in zip(safe_tss_symbols,safe_tss_ids) if sym == symbol]

def tss_id_to_pos(id):
    return safe_tss_pos[tss_pos_ids.index(id)]

def is_between(list_sup,list_sub):
    #Big list first small list later!
    return min(list_sub) >= min(list_sup) and max(list_sub) <= max(list_sup)

In [ ]:
check_chrom = '1'
temp = []
for line in tad_file:
    current_line = line.split('\t')[:3]
    bin_values = (
        int(int(current_line[1]) / RESOLUTION), #Start
        int(int(current_line[2]) / RESOLUTION), #End
        0                                       #Cen
    )
    if check_chrom != current_line[0]:
        with open(tad_path.format(check_chrom), 'wt', newline ='') as file:
            writer = csv.writer(file, delimiter='\t')
            writer.writerow(i for i in header)
            for j in temp:
                writer.writerow(j)
        print(check_chrom,'Done')
        check_chrom = current_line[0]
        temp = [bin_values]
        

    else:
        temp.append(bin_values)
        

In [ ]:
pairs_by_chrom = {}
for chrom in CHROMOSOMES:
    pairs_by_chrom[chrom] = []

prad = pd.read_csv(sqtl_path,delimiter=';')
prad_snp_pos = prad['SNP position'].tolist()
prad_gene_symbols = prad['Gene'].tolist()

tss_ids_df = pd.read_csv(tss_path,delimiter = '\t')
tss_ids_df.dropna(subset = ["Gene_symbol"], inplace=True)
safe_tss_symbols = tss_ids_df['Gene_symbol'].tolist()
safe_tss_ids = tss_ids_df['#refTSSID'].tolist()

tss_pos_df = pd.read_csv(annot_path,delimiter = '\t',header = None)
tss_pos_chroms, safe_tss_pos, tss_pos_ids = tss_pos_df[0].tolist(),tss_pos_df[1].tolist(),tss_pos_df[3].tolist()

def symbol_to_tss_id(symbol):
    return [id for (sym,id) in zip(safe_tss_symbols,safe_tss_ids) if sym == symbol]

def tss_id_to_pos(id):
    return safe_tss_pos[tss_pos_ids.index(id)]
    
symbol_snp_pos = {}
for snp_pos,gene_symbol in zip(prad_snp_pos,prad_gene_symbols):
    if gene_symbol not in symbol_snp_pos.keys():
        symbol_snp_pos[gene_symbol] = [int(snp_pos.split(':')[1])]
    else:
        symbol_snp_pos[gene_symbol].append(int(snp_pos.split(':')[1]))

pairs = {}
count = 0
for gene_symbol_key in symbol_snp_pos:
    tss_ids =  symbol_to_tss_id(gene_symbol_key)
    tss_positions = [tss_id_to_pos(tss_id) for tss_id in tss_ids]
    pairs[gene_symbol_key] = list(itertools.product(tss_positions,symbol_snp_pos[gene_symbol_key]))
    count += 1
    print(gene_symbol_key,'pairs created\t',round(count/4990*100,2),'%')

chrom_genes = {}
for pos,gene in zip(prad_snp_pos,prad_gene_symbols):
    if gene not in chrom_genes.keys():
        chrom_genes[gene] = pos.split(':')[0]

count = 0
total_size = len(pairs)
for gene in pairs:
    for pair in pairs[gene]:
        pairs_by_chrom[chrom_genes[gene]].append(pair)
    count += 1
    print(gene,'pairs added to the result\t',round(count/total_size*100,2),'%')


with open(pair_path, "w") as outfile: 
    json.dump(pairs_by_chrom, outfile)

In [ ]:
with open(pair_path) as data_file:
    data = json.load(data_file)

#Create result dictionary empty
result_by_chrom = {}
for chrom in CHROMOSOMES:
    result_by_chrom[chrom] = [[0,0],[0,0]]

for key in data:
    try:
        tad_data = pd.read_csv(chrom_tad_path.format(key),delimiter='\t')
        tad_list = np.array([tad_data['Start'].tolist(),tad_data['End'].tolist()]).T.tolist()

        true_pairs = [(int(pair[0]/RESOLUTION),int(pair[1]/RESOLUTION)) for pair in data[key]]  #(a,b) true (3,5)
        pseudo_pairs = [(pair[1],pair[1]-pair[0]+pair[1]) for pair in true_pairs]   #(b,b - a + b) pseudo (5,7)

        count = 0
        total_size = len(true_pairs)

        for pair in true_pairs:
            not_found = True
            count += 1
            print(key,'True Pairs\t\t',round(count/total_size*100,2),'%')
            for tad in tad_list:
                if is_between(tad,pair):
                    result_by_chrom[key][0][0] += 1 #TrueYes
                    not_found = False
                    break
            if not_found:
                result_by_chrom[key][0][1] += 1 #TrueNo

        count = 0
        for pair in pseudo_pairs:
            not_found = True
            count += 1
            print(key,'Pseudo Pairs\t\t',round(count/total_size*100,2),'%')
            for tad in tad_list:
                if is_between(tad,pair):
                    result_by_chrom[key][1][0] += 1 #PseudoYes
                    not_found = False
                    break
            if not_found:
                result_by_chrom[key][1][1] += 1 #PseudoNo
    except FileNotFoundError:
        print('TAD for',key,'does not exist')
        
            
            
print('Started Saving')
with open(cont_path, "w") as outfile: 
    json.dump(result_by_chrom, outfile)
print('DONE')

In [ ]:
with open(cont_path) as json_file:
    matrices = json.load(json_file)
    
result_file = open(result_path,'w')
for chrom in CHROMOSOMES:
    try:
        cont_matrix = matrices[chrom]
        print(chrom,'Contingency Matrix:','\n',cont_matrix[0],'\n',cont_matrix[1],'\n---',file = result_file)
        print(chrom,'Mcnemar Results:\n---',file = result_file)
        print(mcnemar(cont_matrix),'\n---',file = result_file)
        oddsr, p = fisher_exact(cont_matrix)
        print(chrom,'Fisher Exact Results:\n---',file = result_file)
        print('Odds Ratio\t',oddsr,file = result_file)
        print('pvalue fish\t',p,file = result_file)
        print('\n--------\n',file = result_file)
    except KeyError:
        print(chrom,'doesnt exist\n---')
json_file.close()
result_file.close()